<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sodapy

In [1]:
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline


In [4]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=200000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
0,8be8e106d741be9ccc1109471c64f58b800b56a5,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,519,1.677496,6,5,0,2,2.55,4.55,True,1,41.9442266014,-87.6559981815,"{'type': 'Point', 'coordinates': [-87.65599818...",41.9477915865,-87.6838349425,"{'type': 'Point', 'coordinates': [-87.68383494...",NaN,NaN
1,bc93948b060228e0e8b0980c14015f85b9421763,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,768,7.513211,NaN,21,5,0,2.55,7.55,True,1,NaN,NaN,NaN,41.9386661962,-87.7112105933,"{'type': 'Point', 'coordinates': [-87.71121059...",NaN,NaN
2,5a9fe836613ba7b344be446ce45cc6deb7495d94,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,743,3.902438,66,56,7.5,0,2.55,10.05,True,1,41.7718485152,-87.695666342,"{'type': 'Point', 'coordinates': [-87.69566634...",41.7925923603,-87.7696154528,"{'type': 'Point', 'coordinates': [-87.76961545...",NaN,NaN
3,675007f893392bd5f5db0c9054fcfdbd478b2cee,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,464,1.643566,1,1,5,0,2.55,7.55,False,1,42.0096228806,-87.6701668569,"{'type': 'Point', 'coordinates': [-87.67016685...",42.0096228806,-87.6701668569,"{'type': 'Point', 'coordinates': [-87.67016685...",NaN,NaN
4,8f3e74e5bd48c79c5d03e8f814450682f74d656f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,723,7.18546,5,8,10,0,2.55,12.55,False,1,41.9477915865,-87.6838349425,"{'type': 'Point', 'coordinates': [-87.68383494...",41.899602111,-87.6333080367,"{'type': 'Point', 'coordinates': [-87.63330803...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,e9d93915090eccbdaefaeaa6cf918bbdd4e148a3,2019-09-30T06:00:00.000,2019-09-30T06:15:00.000,1074,7.0190401032,31,57,12.5,0,2.55,15.05,False,1,41.8502663663,-87.667569312,"{'type': 'Point', 'coordinates': [-87.66756931...",41.8108790078,-87.7263633246,"{'type': 'Point', 'coordinates': [-87.72636332...",NaN,NaN
199996,0e91bf91e667cd950269e9f03846d11f12b6751c,2019-09-30T06:00:00.000,2019-09-30T06:00:00.000,172,0.70594329856,1,1,10,0,2.85,12.85,False,1,42.0096228806,-87.6701668569,"{'type': 'Point', 'coordinates': [-87.67016685...",42.0096228806,-87.6701668569,"{'type': 'Point', 'coordinates': [-87.67016685...",NaN,NaN
199997,08107e35ac56661b7d5aee0f079bccade2b87ab9,2019-09-30T06:00:00.000,2019-09-30T06:30:00.000,1325,7.704372,20,32,15,0,2.55,17.55,False,1,41.9243470769,-87.7347397536,"{'type': 'Point', 'coordinates': [-87.73473975...",41.8788655841,-87.6251921424,"{'type': 'Point', 'coordinates': [-87.62519214...",NaN,NaN
199998,d0cf1453bb4be0b222ec10e6ee3fb74fa9b47ef9,2019-09-30T06:00:00.000,2019-09-30T06:30:00.000,1336,3.3495020944,71,70,10,0,2.55,12.55,False,1,41.7442051463,-87.6563059862,"{'type': 'Point', 'coordinates': [-87.65630598...",41.7457577128,-87.7083657043,"{'type': 'Point', 'coordinates': [-87.70836570...",NaN,NaN


In [2]:
results_df.head()

NameError: name 'results_df' is not defined

In [53]:
results_df.columns

Index(['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'dropoff_community_area', 'fare',
       'tip', 'additional_charges', 'trip_total', 'shared_trip_authorized',
       'trips_pooled', 'pickup_centroid_latitude', 'pickup_centroid_longitude',
       'pickup_centroid_location', 'dropoff_centroid_latitude',
       'dropoff_centroid_longitude', 'dropoff_centroid_location',
       'pickup_census_tract', 'dropoff_census_tract'],
      dtype='object')

In [64]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total
0,8be8e106d741be9ccc1109471c64f58b800b56a5,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,519,1.677496,6,0,2,2.55,4.55
1,bc93948b060228e0e8b0980c14015f85b9421763,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,768,7.513211,NaN,5,0,2.55,7.55
2,5a9fe836613ba7b344be446ce45cc6deb7495d94,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,743,3.902438,66,7.5,0,2.55,10.05
3,675007f893392bd5f5db0c9054fcfdbd478b2cee,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,464,1.643566,1,5,0,2.55,7.55
4,8f3e74e5bd48c79c5d03e8f814450682f74d656f,2019-09-30T23:45:00.000,2019-10-01T00:00:00.000,723,7.18546,5,10,0,2.55,12.55


In [ ]:
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float)  

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 10 columns):
trip_id                  200000 non-null object
trip_start_timestamp     200000 non-null datetime64[ns]
trip_end_timestamp       200000 non-null datetime64[ns]
trip_seconds             200000 non-null int64
trip_miles               200000 non-null float64
pickup_community_area    185706 non-null object
fare                     200000 non-null float64
tip                      200000 non-null float64
additional_charges       200000 non-null float64
trip_total               200000 non-null float64
dtypes: datetime64[ns](2), float64(5), int64(1), object(2)
memory usage: 15.3+ MB


In [ ]:
df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 14 columns):
trip_id                   200000 non-null object
trip_start_timestamp      200000 non-null datetime64[ns]
trip_end_timestamp        200000 non-null datetime64[ns]
trip_seconds              200000 non-null int64
trip_miles                200000 non-null float64
pickup_community_area     185706 non-null object
dropoff_community_area    183628 non-null object
fare                      200000 non-null float64
tip                       200000 non-null float64
additional_charges        200000 non-null float64
trip_total                200000 non-null float64
start_weekday             200000 non-null int64
start_hour                200000 non-null int64
start_time_block          200000 non-null int64
dtypes: datetime64[ns](2), float64(5), int64(4), object(3)
memory usage: 21.4+ MB


In [59]:
columns_to_use = ['start_weekday', 'start_hour', 'pickup_community_area']

columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
X_cat = df.drop(columns=columns_to_drop)
X_cat = X_cat.fillna('0')
X_cat

,pickup_community_area,start_weekday,start_hour
0,6,0,23
1,0,0,23
2,66,0,23
3,1,0,23
4,5,0,23
...,...,...,...
199995,31,0,6
199996,1,0,6
199997,20,0,6
199998,71,0,6


In [83]:
enc = OneHotEncoder()
# X_cat_ohe = enc.fit_transform(X_cat)
# columns = enc.get_feature_names(input_features=X_cat.columns)
# X_cat_df = pd.DataFrame(X_cat_ohe.todense(), columns=columns)
y = df['trip_total']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.25, random_state=12)

# Transform training and test sets
X_train_enc = enc.fit_transform(X_train, y_train)
X_test_enc = enc.transform(X_test)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train.columns)
X_train_df = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
X_test_df = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

X_train_df = sm.add_constant(X_train_df)
model = sm.OLS(y_train,X_train_df).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     713.5
Date:                Tue, 21 Jan 2020   Prob (F-statistic):               0.00
Time:                        20:40:42   Log-Likelihood:            -5.6427e+05
No. Observations:              150000   AIC:                         1.129e+06
Df Residuals:                  149905   BIC:                         1.130e+06
Df Model:                          94                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
pickup_community_area_0     14.3966      0.116    124.441      0.000      14.170      14.623
pickup_community_area_1      0.0818      0.264      0.310      0.757      -0.435       0.599
pickup_community_area_10     0.5871      0.511      1.148      0.251      -0.415       1.589
pickup_community_area_11     1.3834      0.495      2.795      0.005       0.413       2.354
pickup_community_area_12     2.0017      0.682      2.935      0.003       0.665       3.339
pickup_community_area_13     0.2851      0.515      0.554      0.580      -0.724       1.295
pickup_community_area_14    -0.1271      0.342     -0.371      0.711      -0.798       0.544
pickup_community_area_15     0.0296      0.330      0.090      0.928      -0.616       0.676
pickup_community_area_16     0.3398      0.285      1.192      0.233      -0.219       0.899
pickup_community_area_17    -0.8917      0.497     -1.794      0.073      -1.866       0.083
pickup_community_area_18     0.0523      0.820      0.064      0.949      -1.555       1.660
pickup_community_area_19    -1.0955      0.309     -3.546      0.000      -1.701      -0.490
pickup_community_area_2     -0.9267      0.298     -3.109      0.002      -1.511      -0.342
pickup_community_area_20    -1.5518      0.490     -3.167      0.002      -2.512      -0.591
pickup_community_area_21    -0.7814      0.282     -2.770      0.006      -1.334      -0.229
pickup_community_area_22    -0.8651      0.169     -5.129      0.000      -1.196      -0.534
pickup_community_area_23    -1.5806      0.307     -5.149      0.000      -2.182      -0.979
pickup_community_area_24    -1.4604      0.130    -11.219      0.000      -1.716      -1.205
pickup_community_area_25    -1.2300      0.271     -4.540      0.000      -1.761      -0.699
pickup_community_area_26    -1.9500      0.551     -3.537      0.000      -3.030      -0.869
pickup_community_area_27    -1.7701      0.431     -4.109      0.000      -2.614      -0.926
pickup_community_area_28    -1.4632      0.107    -13.643      0.000      -1.673      -1.253
pickup_community_area_29    -1.4213      0.336     -4.234      0.000      -2.079      -0.763
pickup_community_area_3      0.3522      0.201      1.749      0.080      -0.043       0.747
pickup_community_area_30    -1.0678      0.327     -3.262      0.001      -1.709      -0.426
pickup_community_area_31    -1.8943      0.256     -7.413      0.000      -2.395      -1.393
pickup_community_area_32     0.5461      0.103      5.311      0.000       0.345       0.748
pickup_community_area_33     0.1493      0.213      0.700      0.484      -0.268       0.567
pickup_community_area_34    -1.8614      0.418     -4.453      0.000      -2.681      -1.042
pickup_community_area_35    -1.5512      0.314     -4.941      0.000      -2.167      -0.936
pickup_communi

In [ ]:
X_test_df = sm.add_constant(X_test_df)
model_test = sm.OLS(y_test,X_train_df).fit()
model.summary()

In [26]:
# run multiple regression using statsmodels
X_cat_df = sm.add_constant(X_cat_df)
model = sm.OLS(y,X_cat_df).fit()
model.summary()


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     966.3
Date:                Tue, 21 Jan 2020   Prob (F-statistic):               0.00
Time:                        19:17:47   Log-Likelihood:            -7.5121e+05
No. Observations:              200000   AIC:                         1.503e+06
Df Residuals:                  199905   BIC:                         1.504e+06
Df Model:                          94                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
start_weekday_0          -1.503e+12   3.85e+12     -0.390      0.696   -9.05e+12    6.05e+12
start_hour_6              1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_7              1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_8              1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_9              1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_10             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_11             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_12             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_13             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_14             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_15             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_16             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_17             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_18             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_19             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_20             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_21             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_22             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
start_hour_23             1.565e+12   4.01e+12      0.390      0.696    -6.3e+12    9.43e+12
pickup_community_area_0  -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_1  -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_10 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_11 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_12 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_13 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_14 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_15 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_16 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_17 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_community_area_18 -6.204e+10   1.59e+11     -0.390      0.696   -3.74e+11     2.5e+11
pickup_communi

In [81]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression

# Fit the model
linreg = LinearRegression()
linreg.fit(X_train_df, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train_df, y_train))
print('Test r^2:', linreg.score(X_test_df, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_df)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_df)))


Training r^2: 0.3090977429532322
Test r^2: 0.32224597470175154
Training MSE: 108.38359892999804
Test MSE: 103.52141227931178


In [3]:
weather_df = pd.read_csv('../data/chicago_weather.csv')
weather_df

,apparentTemperature,cloudCover,date,dewPoint,hour,humidity,icon,ozone,precipAccumulation,precipIntensity,...,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,50.60,0.98,2018-11-01,40.06,0,0.67,cloudy,290.8,NaN,0.0014,...,rain,1014.7,Overcast,50.60,1541048400,0,10.0,14,4.23,3.37
1,50.47,1.00,2018-11-01,39.10,1,0.65,cloudy,289.5,NaN,0.0019,...,rain,1014.1,Overcast,50.47,1541052000,0,10.0,325,5.31,3.44
2,48.09,1.00,2018-11-01,38.92,2,0.67,cloudy,289.1,NaN,0.0012,...,rain,1014.1,Overcast,49.54,1541055600,0,10.0,345,7.00,4.35
3,47.39,1.00,2018-11-01,38.98,3,0.66,cloudy,289.0,NaN,0.0015,...,rain,1013.8,Overcast,49.79,1541059200,0,10.0,6,11.31,6.06
4,46.49,0.99,2018-11-01,38.13,4,0.64,cloudy,289.1,NaN,0.0005,...,rain,1013.3,Overcast,49.93,1541062800,0,10.0,20,13.13,8.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10220,17.97,0.91,2019-12-31,20.32,19,0.73,cloudy,354.5,NaN,0.0000,...,NaN,1010.1,Overcast,27.88,1577840400,0,10.0,212,17.40,11.25
10221,16.59,0.78,2019-12-31,20.64,20,0.73,partly-cloudy-night,356.8,0.0063,0.0006,...,snow,1011.4,Mostly Cloudy,28.07,1577844000,0,10.0,310,17.45,14.90
10222,16.31,0.64,2019-12-31,20.58,21,0.75,partly-cloudy-night,358.7,NaN,0.0000,...,NaN,1011.1,Mostly Cloudy,27.40,1577847600,0,10.0,272,16.62,13.53
10223,16.86,0.68,2019-12-31,20.83,22,0.80,partly-cloudy-night,359.4,0.0079,0.0007,...,snow,1011.2,Mostly Cloudy,26.27,1577851200,0,10.0,276,15.84,9.59
